# Plot XRD maps from flyscans

This notebook will plot an XRD map from a flyscan. It requires data from the `.h5` masterfile and the `_pilatus_integrated.h5` file.

A ROI for the XRD map (in radial bins) should be defined in one of the cell below.

### Load data and plot average XRD for the map

In [1]:
%matplotlib widget
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import DanMAX as DM

# Define scan location:
scan = 'XXXX'
folder = '/data/visitors/danmax/PROPOSAL/VISIT/raw/SAMPLE/'

# Define the ROI for the XRD map
radial_roi = [500,550]

fh = h5py.File(os.path.join(folder, 'scan-%s.h5' %scan), 'r')
print("Scan commmand used: {0}".format(fh['entry/title'][()].decode('utf-8')))
macroname = fh['entry/title'][()].decode('utf-8').split()
nominalYsteps = int(macroname[4])+1
nominalXsteps = int(macroname[8])+1

print('original scan dimensions are '+str(nominalXsteps)+'x'+str(nominalYsteps)+' points')

# first motor in meshct is the fast one
x = fh['/entry/instrument/ea_ppm3_3/value'][:]
#y = fh['/entry/instrument/ea_ppm3_2/value'][:]
y = fh['/entry/instrument/pcap_mean/data'][:]

#if the scan was interrupted, get the actual dimension in X, to build a map of the correct aspect ratio
actualXsteps = int(len(x)/nominalYsteps)
print('originally '+str(nominalXsteps)+' horizontal steps were plannes')
print('actually '+str(actualXsteps)+' full horizontal steps are found in the file')

def make_map(x, y, signal):
    xmin, xmax = np.amin(x), np.amax(x)
    ymin, ymax = np.amin(y), np.amax(y)

    xi = np.linspace(xmin, xmax, 2*actualXsteps)
    yi = np.linspace(ymin, ymax, 2*nominalYsteps)

    XI, YI = np.meshgrid(xi, yi)
    ZI = griddata((x.reshape(-1), y.reshape(-1)), signal.reshape(-1), (XI, YI), 'nearest')
    return ZI

fh = h5py.File(os.path.join(folder.replace('raw', 'process/azint'), 'scan-%s_pilatus_integrated.h5' %scan), 'r')

plt.figure()
plt.plot(np.mean(fh['I'][:], axis=0))
plt.axvline(radial_roi[0], color='black', linestyle='--')
plt.axvline(radial_roi[1], color='black', linestyle='--')

Matplotlib is building the font cache; this may take a moment.


ModuleNotFoundError: No module named 'Ipython'

### Calculate XRD map
This may take a while...

In [ ]:
signal = np.empty(fh['I'].shape[0])
for i, shot in enumerate(fh['I']):
    signal[i] = np.mean(shot[radial_roi[0]:radial_roi[1]])
    
img = DM.makeMap(x, y, actualXsteps, nominalYsteps, signal)
print('Map ready!')

### Plot XRD map
It is probably nescessesary to change the vmax value

In [ ]:
plt.figure()
plt.imshow(img[:,:], aspect=1, vmax=0.5, cmap='viridis', extent=[x[0], x[-1], y[0], y[-1]])
plt.xlabel('ea_ppm3_3 / mm')
plt.ylabel('pcap_mean (ea_ppm3_2) / mm')

#usefull cmaps:
# 'CMRmap', 'GnBu', 'OrRd', PuBu', 'PuBuGn', 'PuOr', 'PuRd',
# 'RdPu', 'RdYlGn', 'Reds', 'Spectral', 'Wistia', 'YlOrRd', 'autumn',
# 'cividis', 'coolwarm', 'copper', 'gist_earth', 'gist_heat', 'gnuplot',
# 'hot', 'inferno', 'jet', 'magma', 'nipy_spectral', 'ocean', 'plasma', 'rainbow',
# 'seismic', 'spring', 'summer', 'viridis', 'winter'